In [2]:
"""
Zonal Statistics - Calcula estadisticas por distrito
Ejecuta directamente, genera CSV y muestra tabla
"""

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import pandas as pd
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

### Configurar rutas
BASE_DIR = Path("C:/Users/ASUS/OneDrive - Universidad del Pacífico/Tareas Data Science/Minimum-Temperature-Raster")
DATA_DIR = BASE_DIR / 'data'
PROCESSED_DIR = DATA_DIR / 'processed'
OUTPUT_DIR = DATA_DIR / 'outputs'
TABLES_DIR = OUTPUT_DIR / 'tables'
TABLES_DIR.mkdir(parents=True, exist_ok=True)

print("=" * 70)
print("PASO 2: ESTADISTICAS ZONALES")
print("=" * 70)

### CARGAR DATOS PROCESADOS
print("\n[1/4] Cargando datos procesados...")

districts = gpd.read_file(PROCESSED_DIR / 'peru_districts.gpkg')
metadata = pd.read_csv(PROCESSED_DIR / 'raster_metadata.csv')
scale_factor = metadata['scale_factor'].iloc[0]

print(f"Distritos: {len(districts)}")
print(f"Factor de escala: {scale_factor}")

### CALCULAR ESTADISTICAS ZONALES
print("\n[2/4] Calculando estadisticas zonales...")

raster_path = DATA_DIR / 'raw' / 'tmin_raster.tif'
results = []

with rasterio.open(raster_path) as src:
    ### Reproyectar si necesario
    if districts.crs != src.crs:
        print("Reproyectando distritos...")
        districts = districts.to_crs(src.crs)
    
    total = len(districts)
    for idx, district in districts.iterrows():
        try:
            ### Enmascarar raster con geometría del distrito
            geom = [district.geometry.__geo_interface__]
            masked_data, _ = mask(src, geom, crop=True, nodata=np.nan)
            
            ### Promediar bandas si es multiband
            if masked_data.ndim == 3 and masked_data.shape[0] > 1:
                data = np.nanmean(masked_data, axis=0)
            else:
                data = masked_data[0]
            
            ### Aplicar escala y filtrar validos
            data = data * scale_factor
            valid = data[~np.isnan(data)]
            
            if len(valid) > 0:
                ### Estadisticas basicas (6 metricas minimas + 1 custom)
                mean_val = np.mean(valid)
                min_val = np.min(valid)
                std_val = np.std(valid)
                
                stats = {
                    'UBIGEO': district['UBIGEO'],
                    'NAME': district['NAME'],
                    'REGION': district['REGION'],
                    'count': len(valid),
                    'mean': mean_val,
                    'min': min_val,
                    'max': np.max(valid),
                    'std': std_val,
                    'p10': np.percentile(valid, 10),
                    'p90': np.percentile(valid, 90),
                }
                
                ### METRICA PERSONALIZADA: Indice de Riesgo de Heladas
                ### Componentes: 40% mean, 35% min, 25% variabilidad
                mean_risk = max(0, (10 - mean_val) / 10) * 40
                min_risk = max(0, (5 - min_val) / 10) * 35
                var_risk = min(std_val / 5, 1) * 25
                
                stats['frost_risk_index'] = mean_risk + min_risk + var_risk
                
                ### Categoria de riesgo
                fri = stats['frost_risk_index']
                if fri >= 75:
                    stats['risk_category'] = 'Very High'
                elif fri >= 50:
                    stats['risk_category'] = 'High'
                elif fri >= 25:
                    stats['risk_category'] = 'Moderate'
                else:
                    stats['risk_category'] = 'Low'
                
                results.append(stats)
            
            ### Progreso
            if (idx + 1) % 100 == 0:
                print(f"  Procesados: {idx + 1}/{total}")
                
        except Exception as e:
            continue

### Convertir a DataFrame
stats_df = pd.DataFrame(results)

print(f"\nEstadisticas calculadas: {len(stats_df)} distritos")

### MOSTRAR RESUMEN
print("\n[3/4] Resumen de resultados:")

print(f"\nTemperatura media general: {stats_df['mean'].mean():.2f} C")
print(f"Temperatura minima registrada: {stats_df['min'].min():.2f} C")
print(f"Temperatura maxima registrada: {stats_df['max'].max():.2f} C")

print(f"\nDistribucion por categoria de riesgo:")
risk_counts = stats_df['risk_category'].value_counts()
for category, count in risk_counts.items():
    pct = (count / len(stats_df)) * 100
    print(f"  {category}: {count} ({pct:.1f}%)")

### MOSTRAR TOP 10 RIESGO
print(f"\nTop 10 distritos de MAYOR riesgo:")
top_risk = stats_df.nlargest(10, 'frost_risk_index')[['NAME', 'REGION', 'mean', 'min', 'frost_risk_index']]
print(top_risk.to_string(index=False))

### GUARDAR RESULTADOS
print("\n[4/4] Guardando resultados...")

### CSV completo
csv_path = TABLES_DIR / 'zonal_statistics.csv'
stats_df.to_csv(csv_path, index=False)
print(f"Guardado: {csv_path.name}")

### Excel con multiples hojas
excel_path = TABLES_DIR / 'zonal_statistics.xlsx'
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    stats_df.to_excel(writer, sheet_name='All_Districts', index=False)
    
    ### Por categoria de riesgo
    for category in ['Very High', 'High', 'Moderate', 'Low']:
        subset = stats_df[stats_df['risk_category'] == category]
        if len(subset) > 0:
            sheet_name = category.replace(' ', '_')
            subset.to_excel(writer, sheet_name=sheet_name, index=False)
    
    ### Top 50 riesgo
    top50 = stats_df.nlargest(50, 'frost_risk_index')
    top50.to_excel(writer, sheet_name='Top_50_Risk', index=False)

print(f"Guardado: {excel_path.name}")

### Guardar resumen
summary = pd.DataFrame([{
    'total_districts': len(stats_df),
    'mean_temp_overall': stats_df['mean'].mean(),
    'min_temp_overall': stats_df['min'].min(),
    'max_temp_overall': stats_df['max'].max(),
    'very_high_risk': len(stats_df[stats_df['risk_category'] == 'Very High']),
    'high_risk': len(stats_df[stats_df['risk_category'] == 'High']),
    'moderate_risk': len(stats_df[stats_df['risk_category'] == 'Moderate']),
    'low_risk': len(stats_df[stats_df['risk_category'] == 'Low'])
}])

summary_path = TABLES_DIR / 'summary_statistics.csv'
summary.to_csv(summary_path, index=False)
print(f"Guardado: {summary_path.name}")

print("\n" + "=" * 70)
print("ESTADISTICAS ZONALES COMPLETADAS")
print("=" * 70)

PASO 2: ESTADISTICAS ZONALES

[1/4] Cargando datos procesados...
Distritos: 1873
Factor de escala: 1.0

[2/4] Calculando estadisticas zonales...
  Procesados: 100/1873
  Procesados: 200/1873
  Procesados: 300/1873
  Procesados: 400/1873
  Procesados: 500/1873
  Procesados: 600/1873
  Procesados: 700/1873
  Procesados: 800/1873
  Procesados: 900/1873
  Procesados: 1000/1873
  Procesados: 1100/1873
  Procesados: 1200/1873
  Procesados: 1300/1873
  Procesados: 1400/1873
  Procesados: 1500/1873
  Procesados: 1600/1873
  Procesados: 1700/1873
  Procesados: 1800/1873

Estadisticas calculadas: 1789 distritos

[3/4] Resumen de resultados:

Temperatura media general: 9.45 C
Temperatura minima registrada: -8.86 C
Temperatura maxima registrada: 23.72 C

Distribucion por categoria de riesgo:
  Low: 1038 (58.0%)
  Moderate: 409 (22.9%)
  High: 250 (14.0%)
  Very High: 92 (5.1%)

Top 10 distritos de MAYOR riesgo:
      NAME   REGION      mean       min  frost_risk_index
  SUSAPAYA    TACNA -5.157803